#Task 
Predict ว่าคนหนึ่งๆจะเรียนจบคอร์สหนึ่งๆหรือไม่
#Dataset (df2)
  ใน Dataframe แต่ละแถวจะมี cv_uid,cv_cid ซ้ำกันได้ แต่จะไม่ซ้ำกันทั้งสองตัว ผมใช้ 6 Feature คือ
*   complete_count = จำนวนคอร์สที่เรียนจบ ก่อนจะมา enroll คอร์สนี้
*   enroll_count= จำนวนคอร์สที่ enroll ก่อนจะมา enroll คอร์สนี้
*   avg_score = คะแนนเฉลี่ย (คิดเป็นร้อยละ/100) ของคอร์สนี้
*   quiz_submit=จำนวนครั้งที่ส่ง quiz คอร์สนี้
*   quiz_open=จำนวนครั้งที่เปิด quiz คอร์สนี้
*   did_watch=เช็คว่าเคยดูวิดีโอของคอร์สนี้หรือไม่ ถ้าเคยจะเป็น 1 ถ้าไม่เคยจะเป็น 0
* G1,G2,G3=boolean features G1=1 เมื่อนักเรียนเป็น Male G2=1 เมื่อนักเรียนเป็น Female G3=1 เมื่อนักเรียนไม่ใช่ทั้ง Male และ Female




ผมทำ Stacking 2 Layer
Layer 1 : 


*   Logistic Regression
*   LigthGBM
*   Adaboost
*   Extratree
*   K-nearest neighbor
*   Random Forest
*   XGBoost
*   Neural Network

#ขั้นตอน



1.   เอา Data ทั้งหมด Train ให้แต่ละ model แล้วเอา Predict Probability ของแต่ละ model รวมกับ feature เดิม เอามาทำเป็น Dataset ใหม่ 




Layer2 : ใช้ตัวเดียวคือ LogisticRegression ใช้ Train กับ Dataset ใหม่ที่ได้จาก Layer1 

In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.metrics import classification_report




In [0]:
student=pd.read_csv("05_cvocp_user_info_masked.csv")
df2["G1"]=0
df2["G2"]=0
df2["G3"]=0
for idx,row in student.iterrows():
  if row["gender"]=="Male":
    df2.loc[df2["cv_uid"]==row["cv_uid"],["G1"]]=1
  elif row["gender"]=="Female":
    df2.loc[df2["cv_uid"]==row["cv_uid"],["G2"]]=1
  else:
    df2.loc[df2["cv_uid"]==row["cv_uid"],["G3"]]=1
  print(idx)


In [0]:
#df2=pd.read_csv("20191112_CUMOOC_Data/dfneweraaisus.csv")
df2=pd.read_csv("dfneweraaisus.csv")
df2=df2.drop(columns=["all_score","nub","cv_uid","cv_cid"])


In [0]:
df2 = shuffle(df2)
X_train2=pd.DataFrame(df2)
Y_train2=X_train2["label"]
XOK=X_train2[85000:]
XOK=XOK[XOK["label"]==1]
YOK=XOK["label"]
XOK=XOK.drop(columns=["label"])
X_train2=X_train2.drop(columns=["label"])
Layer1DF=pd.DataFrame(df2)

In [0]:
df2.to_csv("df_sex.csv",index=False)


#ลองเอา dataset มา Train ก่อน

In [0]:
lg=LogisticRegression()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)


0.8918675929709848
0.763271162123386 0.7140939597315437 0.737864077669903


In [0]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=6))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2[:85000],Y_train2[:85000])
k=KRlayer1.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=KRlayer1.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





85000/85000 [==============================] - 35s 407us/step - loss: 0.2663 - acc: 0.8775
Epoch 2/100
85000/85000 [==============================] - 25s 293us/step - loss: 0.2288 - acc: 0.8953
Epoch 3/100
85000/85000 [==============================] - 25s 297us/step - loss: 0.2224 - acc: 0.8980
Epoch 4/100
85000/85000 [==============================] - 26s 303us/step - loss: 0.2172 - acc: 0.9008
Epoch 5/100
85000/85000 [==============================] - 25s 292us/step - loss: 0.2156 - acc: 0.9022
Epoch 6/100
85000/85000 [==============================] - 25s 293us/step - loss: 0.2141 - acc: 0.9016
Epoch 7/100
85000/85000 [==============================] - 25s 290us/step - loss: 0.2133 - acc: 0.9019
Epoch 8/100
85000/85000 [==============================]

In [0]:
lg=XGBClassifier()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)
print(classification_report(Y_train2[85000:], lgy))

0.9409889660809154
0.8408342480790341 0.8889748549323018 0.864234674689733
              precision    recall  f1-score   support

           0       0.97      0.95      0.96     19300
           1       0.84      0.89      0.86      5170

    accuracy                           0.94     24470
   macro avg       0.91      0.92      0.91     24470
weighted avg       0.94      0.94      0.94     24470



In [0]:
lg=XGBClassifier()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(XOK, YOK)
print(k)
lgy=lg.predict(XOK)
tn, fp, fn, tp = confusion_matrix(YOK,lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

0.8889748549323018
1.0 0.8889748549323018 0.9412246569731723


#Stacking

In [0]:
LRlayer1=LogisticRegression()
LRlayer1.fit(X_train2,Y_train2)
out=LRlayer1.predict_proba(X_train2)
Layer1DF["LRProb"]=out[:,1]
Layer1DF

,label,enroll_count,complete_count,quiz_submit,quiz_open,avg_score,did_watch,G1,G2,G3,LRProb
68257,0,3,0,1,2,0.0,1,0,1,0,0.137625
88523,0,17,1,0,0,0.0,0,0,1,0,0.003002
37395,0,5,0,0,0,0.0,0,0,1,0,0.008142
99090,1,2,2,10,16,0.7,0,0,0,0,0.586932
45208,0,1,0,1,1,0.0,0,0,1,0,0.013391
...,...,...,...,...,...,...,...,...,...,...,...
7786,0,2,0,0,0,0.0,0,0,1,0,0.011173
36783,0,10,3,0,1,0.0,0,1,0,0,0.011409
17257,0,5,0,0,0,0.0,0,0,1,0,0.008142
68156,0,1,0,1,1,0.0,1,1,0,0,0.156000


In [0]:
lg=lgbm.LGBMClassifier()
lg.fit(X_train2,Y_train2)
out=lg.predict_proba(X_train2)
Layer1DF["LGProb"]=out[:,1]

In [0]:
ada=AdaBoostClassifier()
ada.fit(X_train2,Y_train2)
out=ada.predict_proba(X_train2)
Layer1DF["ADAProb"]=out[:,1]


In [0]:
ext=ExtraTreesClassifier()
ext.fit(X_train2,Y_train2)
out=ext.predict_proba(X_train2)
Layer1DF["EXTProb"]=out[:,1]


In [0]:
kn=KNeighborsClassifier()
kn.fit(X_train2,Y_train2)
out=kn.predict_proba(X_train2)
Layer1DF["KNProb"]=out[:,1]

In [0]:
rf=RandomForestClassifier()
rf.fit(X_train2, Y_train2)
out=rf.predict_proba(X_train2)
Layer1DF["RFProb"]=out[:,1]



In [0]:
xgb = XGBClassifier()
xgb.fit(X_train2, Y_train2)
print(xgb.score(X_train2,Y_train2))
out=xgb.predict_proba(X_train2)
Layer1DF["XGProb"]=out[:,1]
#OO=xgb.feature_importances_.reshape(1,6)
#for i in range(OO.shape[1]):
#  IM[i]+=OO[0,i]

0.9419384306202613


In [0]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=9))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2,Y_train2)
out=KRlayer1.predict_proba(X_train2)
Layer1DF["KRProb"]=out[:,1]




Epoch 1/100





109470/109470 [==============================] - 56s 512us/step - loss: 0.2445 - acc: 0.8854
Epoch 2/100
109470/109470 [==============================] - 46s 421us/step - loss: 0.2077 - acc: 0.9029
Epoch 3/100
109470/109470 [==============================] - 47s 428us/step - loss: 0.1984 - acc: 0.9070
Epoch 4/100
109470/109470 [==============================] - 47s 426us/step - loss: 0.1915 - acc: 0.9100
Epoch 5/100
109470/109470 [==============================] - 46s 421us/step - loss: 0.1873 - acc: 0.9126
Epoch 6/100
109470/109470 [==============================] - 46s 420us/step - loss: 0.1855 - acc: 0.9143
Epoch 7/100
109470/109470 [==============================] - 46s 422us/step - loss: 0.1851 - acc: 0.9145
Epoch 8/100
109470/109470 [==============================] - 46s 422us/step - loss: 0.1849 - acc: 0.9147
Epoch 9/100
109470/109470 [==============================] - 46s 421us/step - loss: 0.1841 - acc: 0.9153
Epoch 10/100
109470/109470 [===================

คิด Feature Importance จะได้ว่า complete_count กับ did_watch สำคัญน้อยสุดเลยตัดทิ้งไป

In [0]:
#IM=[[-val,key] for key,val in IM.items()]
#IM.sort()
#print(IM)
#Layer1DF=Layer1DF.drop(columns=["complete_count","did_watch"])
#Layer1DF[Layer1DF["label"]==1]
#Layer1DF

In [0]:
Layer1DF.to_csv("dflayer1_sex.csv",index=False)

#Layer2

Layer2 : ใช้ตัวเดียวคือ LogisticRegression (ผมลองใช้ตัวอื่นแล้วไม่ต่างกัน ได้ accuracy ประมาณ 0.97 f1-score ประมาณ 0.92) 










In [0]:
Layer1DF=pd.read_csv("dflayer1_sex.csv")

In [0]:
lg=LogisticRegression()
Y=Layer1DF["label"]
MM=Layer1DF.drop(columns=["label"])
lg.fit(MM[:85000],Y[:85000])
k=lg.score(MM[85000:], Y[85000:])
print(k)
lgy=lg.predict(MM[85000:])
tn, fp, fn, tp = confusion_matrix(Y[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

0.9738863914997957
0.9394928928159816 0.9378715244487057 0.9386815084924672


ผมลอง Stack เพิ่มอีก 1 Layer แล้ว ผลลัพธ์ที่ได้ไม่ต่างกันผมเลย Stack แค่ 2 ชั้น 

